In [1]:
import os
import pandas as pd
import json

## Analisis de estados

En esta sección cargaremos el archivo metadata, este archivo anteriormente fue filtrado y contiene la información de los negoción de **Fast food**. El objetivo de esta sección es analisas el archivo de reviews de estados de los negoción que tenemos registros en el dataset de metadata.

In [4]:
metadata_fast_food_path = os.join.path('../../','data','google','metadata.csv.gz')

df_metadata = pd.read_csv(metadata_fast_food_path, compression='gzip')

In [5]:
df_metadata

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category2
0,Maverik Adventure's First Stop,"Maverik Adventure's First Stop, 3330 S 500 W, ...",0x87528bb7ab938cb3:0x5ee996e34cc50715,NaN,40.698585,-111.906406,['Fuel supplier' 'Convenience store' 'Fast foo...,3.4,5,NaN,"[array(['Thursday', 'Open 24 hours'], dtype=ob...",{'Accessibility': array(['Wheelchair accessibl...,Open 24 hours,NaN,https://www.google.com/maps/place//data=!4m2!3...,['Fuel supplier' 'Convenience store' 'Fast foo...
1,Pop's Express OB,"Pop's Express OB, 10047 MS-178, Olive Branch, ...",0x887f93d5418166af:0xbbce72cb50342df1,NaN,34.949467,-89.812309,['Fast food restaurant' 'Beer store' 'Convenie...,4.4,25,NaN,"[array(['Thursday', '6AM–10PM'], dtype=object)...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,['0x887f91e2983ef1db:0xe7667f8770e744de'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Fast food restaurant' 'Beer store' 'Convenie...
2,Dunkin',"Dunkin', 4008 Bell Blvd, Queens, NY 11361",0x89c261f60bdf13db:0x38da730e4687a97b,Long-running chain serving signature breakfast...,40.763985,-73.771430,['Coffee shop' 'Bagel shop' 'Bakery' 'Breakfas...,3.5,8,$,"[array(['Thursday', '6AM–7PM'], dtype=object)\...",{'Accessibility': array(['Wheelchair accessibl...,Open ⋅ Closes 7PM,['0x89c3ab9229879ec3:0x3f4b2b46d7d2c503'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Coffee shop' 'Bagel shop' 'Bakery' 'Breakfas...
3,Cowboy Burgers & BBQ,"Cowboy Burgers & BBQ, 13101 Ramona Blvd, Baldw...",0x80c2d765f8c90a3d:0x16afb75943e7ad50,"American grub such as BBQ ribs, hamburgers, sa...",34.079995,-117.988951,['Hamburger restaurant' 'American restaurant' ...,3.7,38,NaN,"[array(['Thursday', '6AM–9PM'], dtype=object)\...",{'Accessibility': array(['Wheelchair accessibl...,Closed ⋅ Opens 6AM,['0x80c2d765f8dd4ebf:0xb6baf31e3e536ffa'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Hamburger restaurant' 'American restaurant' ...
4,Kennedy's Chicken & Sandwiches,"Kennedy's Chicken & Sandwiches, 495 E 138th St...",0x89c2f5c3caef3fb5:0xb7f855503b4bc974,NaN,40.808010,-73.919617,['Fast food restaurant'],3.7,8,NaN,"[array(['Thursday', '8AM–3AM'], dtype=object)\...","{'Accessibility': None, 'Activities': None, 'A...",Closed ⋅ Opens 8AM,['0x89c2f5cfbcdeb7c1:0x1c3c27e3a6bcdab6'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Fast food restaurant']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28795,Agua Pura & Snack's,"Agua Pura & Snack's, 13669 Manning Ave #106, P...",0x8094fb3d40ce4ad9:0xfb24e5d4fb2159ce,NaN,36.604000,-119.546084,['Fast food restaurant' 'Crêperie' 'Ice cream ...,4.4,28,NaN,"[array(['Saturday', '12–6PM'], dtype=object)\n...",{'Accessibility': array(['Wheelchair accessibl...,Closed ⋅ Opens 12PM,['0x8094fb22b42ca4bf:0x11ae3f6f770b7629'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Fast food restaurant' 'Crêperie' 'Ice cream ...
28796,Subway,"Subway, 620 N Cedar St Suite B, Mason, MI 48854",0x8822c54163af5d89:0x74cc4e15b662c024,Casual counter-serve chain for build-your-own ...,42.588522,-84.455860,['Sandwich shop' 'Caterer' 'Fast food restaura...,3.9,135,$,"[array(['Saturday', '9AM–10PM'], dtype=object)...",{'Accessibility': array(['Wheelchair accessibl...,Open ⋅ Closes 10PM,['0x8822c541869b557b:0xd0023a2aa6554a22'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Sandwich shop' 'Caterer' 'Fast food restaura...
28797,Bojangles,"Bojangles, 501 S College St, Charlotte, NC 28202",0x8856a028f0b0a1e7:0x7ff78df2deab30,Southern-inspired fast-food chain known for Ca...,35.221936,-80.844894,['Restaurant' 'American restaurant' 'Chicken r...,3.6,98,$,NaN,{'Accessibility': array(['Wheelchair accessibl...,NaN,['0x88569f8ca01bc765:0xcdf7776ade3dfdba'\n '0x...,https://www.google.com/maps/place//data=!4m2!3...,['Restaurant' 'American restaurant' 'Chicken r...
28798,Little Caesars Pizza,"Littl

## Unificación de la información de estados

Tenemos información de las reviews de 52 estados, el objetivo de esta sección es concatenar los 52 estados en un solo archivo csv.gz, solamente de los negoción que tenemos información es decir los negocios que se encuentran en metada.

Por cada estado tenemos varios archivos json con la información de las reviews, el siguiente codigo va a revisar cada una de las 52 carpetas y por cada carpeta va a concatenar estos archivos en uno solo, es claro que esto es una gran cantidad de información, por lo que en el trancurso de la carga vamos ir filtrando por los archivos que tenemos información. Este filtro anterior va a reducir significativamente el peso del archivo, ademas de quedarnos solo con los registros que nos interesa para el negocio.

**Unificar información**

En este primer bloque vamos a concatenar cada uno de los archivos por carpeta.

In [6]:
import os
import pandas as pd

directorio_base = '../Datasets/GoogleMaps/reviews-estados'

# Recorrer todas las carpetas dentro de 'reviews-estados'
for estado in os.listdir(directorio_base):
    estado_path = os.path.join(directorio_base, estado)

    # Verificar si es un directorio
    if os.path.isdir(estado_path):
        dataframes = []  # Lista para almacenar los DataFrames de cada carpeta

        # Recorrer los archivos dentro de la carpeta del estado
        for filename in os.listdir(estado_path):
            if filename.endswith(".json"):
                filepath = os.path.join(estado_path, filename)

                # Leer el archivo JSON en bloques o líneas y convertirlo en DataFrame
                chunk_size = 100000  # ajusta el tamaño del bloque según tu disponibilidad de memoria
                for df_chunk in pd.read_json(filepath, lines=True, orient='records', encoding='utf-8', chunksize=chunk_size):
                    dataframes.append(df_chunk)

        # Concatenar todos los DataFrames en uno solo
        estado_dataframe = pd.concat(dataframes, ignore_index=True)

        # Guardar el DataFrame en un archivo CSV dentro de la carpeta del estado
        csv_filename = os.path.join(estado_path, f'{estado}_reviews.csv')
        estado_dataframe.to_csv(csv_filename, index=False)

**Filtrado por Fast food**

en este siguiente bloque vamos a filtrar por los registros que contienen información de **Fast food** que se encuentran en metadata, este será el archivo filtrado, que posteriormente utilizaremos para tener toda la información en un archivo.

In [9]:

import os
import pandas as pd

directorio_base = '../Datasets/GoogleMaps/reviews-estados'

# Leer el archivo CSV comprimido
metadata_fast_food_path = os.join.path('../../','data','google','metadata.csv.gz')
df_metadata = pd.read_csv(metadata_fast_food_path, compression='gzip')

# Recorrer todas las carpetas dentro de 'reviews-estados'
for estado in os.listdir(directorio_base):
    estado_path = os.path.join(directorio_base, estado)

    # Verificar si es un directorio
    if os.path.isdir(estado_path):
        dataframes = []  # Lista para almacenar los DataFrames de cada carpeta

        # Recorrer los archivos dentro de la carpeta del estado
        for filename in os.listdir(estado_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(estado_path, filename)

                # Leer el archivo CSV
                estado_dataframe = pd.read_csv(filepath)

                # Filtrar por GMap IDs presentes en el archivo CSV comprimido
                estado_dataframe_filtered = estado_dataframe[estado_dataframe['gmap_id'].isin(df_metadata['gmap_id'])]

                # Guardar el DataFrame filtrado en un nuevo archivo CSV
                csv_filtered_filename = os.path.join(estado_path, f'{estado}_reviews_filtered.csv')
                estado_dataframe_filtered.to_csv(csv_filtered_filename, index=False)
